<a href="https://colab.research.google.com/github/LoriSchuan-dev/transferLearning_AIwearable/blob/main/UNT_CSCE5280_Fall2021_TransferLearning_Iteration2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import platform

#!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
#from google.colab import drive
from oauth2client.client import GoogleCredentials

os.getcwd()
os.listdir()
#drive.mount('/content/drive')
#os.getcwd()
#os.listdir("./drive/MyDrive")


['.config', 'sample_data']

In [2]:
# cloning our team's repo on github
!git clone https://github.com/LoriSchuan-dev/transferLearning_AIwearable
os.listdir()

Cloning into 'transferLearning_AIwearable'...
remote: Enumerating objects: 4140, done.
remote: Counting objects: 100% (4140/4140), done.
remote: Compressing objects: 100% (4059/4059), done.
remote: Total 4140 (delta 532), reused 3683 (delta 75), pack-reused 0
Receiving objects: 100% (4140/4140), 65.18 MiB | 26.77 MiB/s, done.
Resolving deltas: 100% (532/532), done.
Checking out files: 100% (9973/9973), done.


['.config', 'transferLearning_AIwearable', 'sample_data']